In [43]:
import cv2 as cv
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [44]:
transform = transforms.Compose(
    [
        transforms.Resize((48, 48)),  # Resize images to 48x48
        transforms.Grayscale(num_output_channels=1),  # Convert images to grayscale
        transforms.ToTensor(),  # Convert images to PyTorch tensors
    ]
)

dataset = datasets.ImageFolder(root="./train", transform=transform)


labels = dataset.targets

# Define the split ratios
train_ratio = 0.7
valid_ratio = 0.15
test_ratio = 0.15
# increase training ratio with more data

# Calculate the number of samples for each set
total_samples = len(dataset)
train_size = int(train_ratio * total_samples)
valid_size = int(valid_ratio * total_samples)
test_size = total_samples - train_size - valid_size

# Create indices for the splits
indices = torch.randperm(total_samples)
train_indices = indices[:train_size]
valid_indices = indices[train_size : train_size + valid_size]
test_indices = indices[train_size + valid_size :]

# Create data loaders for each split
train_loader = torch.utils.data.DataLoader(
    dataset, sampler=SubsetRandomSampler(train_indices)
)
valid_loader = torch.utils.data.DataLoader(
    dataset, sampler=SubsetRandomSampler(valid_indices)
)
test_loader = torch.utils.data.DataLoader(
    dataset, sampler=SubsetRandomSampler(test_indices)
)

In [45]:
class NNet(nn.Module):
    def __init__(self, learning_rate=0.001, batch_size=32):
        super(NNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 48, kernel_size=3)
        self.conv2 = nn.Conv2d(48, 48, kernel_size=3)
        self.pool = nn.MaxPool2d(3, 3)
        self.fc1 = nn.Linear(48 * 4 * 4, 144) # Adjusted output size
        self.fc2 = nn.Linear(144, 48) # Corrected input size to match the output of self.fc1
        self.out = nn.Linear(48, 24)
        self.learning_rate = learning_rate
        self.batch_size = batch_size

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(-1, 48 * 4 * 4) # Corrected reshaping based on the calculated dimensions
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.softmax(self.out(x), dim=1)

In [46]:
# train the network
emotions = ["angry", "disgusted", "fearful", "happy", "neutral", "sad", "surprised"]

cnn = NNet()

cross_entropy_loss = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(cnn.parameters(), lr=0.001, momentum=0.9)

In [47]:
for epoch in range(3):
    total_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        # zero parameter gradients
        optimizer.zero_grad()

        # forward & backward run, SGD
        outputs = cnn(inputs)
        loss = cross_entropy_loss(outputs, labels)
        loss.backward()  # how do we get this?
        optimizer.step()  # optimization step

        # display stats
        total_loss += loss.item()
        if i % 2000 == 1999:  # print every 2K mini-batches
            print(f"[{epoch + 1}, {i+1:5d}] loss: {loss/2000:.3f}")
            loss = 0.0  # reset loss

print("Training done")

[1,  2000] loss: 0.002
[1,  4000] loss: 0.001
[1,  6000] loss: 0.001
[1,  8000] loss: 0.002
[1, 10000] loss: 0.002
[1, 12000] loss: 0.002
[1, 14000] loss: 0.002
[1, 16000] loss: 0.002
[1, 18000] loss: 0.002
[1, 20000] loss: 0.002
[2,  2000] loss: 0.001
[2,  4000] loss: 0.001
[2,  6000] loss: 0.001
[2,  8000] loss: 0.002
[2, 10000] loss: 0.002
[2, 12000] loss: 0.001
[2, 14000] loss: 0.002
[2, 16000] loss: 0.002
[2, 18000] loss: 0.002
[2, 20000] loss: 0.002
[3,  2000] loss: 0.002
[3,  4000] loss: 0.002
[3,  6000] loss: 0.002
[3,  8000] loss: 0.002
[3, 10000] loss: 0.002
[3, 12000] loss: 0.001
[3, 14000] loss: 0.001
[3, 16000] loss: 0.002
[3, 18000] loss: 0.002
[3, 20000] loss: 0.002
Training done


In [48]:
# OpenCV Functions


# Show image
def show_img(img):
    cv.imshow(img, cv.imread(img))
    cv.waitKey(0)
    cv.destroyWindow(img)


# Grayscale
def show_grayscale(img):
    gray = cv.cvtColor(cv.imread(img), cv.COLOR_BGR2GRAY)
    cv.imshow(img, gray)
    cv.waitKey(0)
    cv.destroyWindow(img)


# Edge detection
def show_canny(img):
    canny = cv.Canny(cv.imread(img), 100, 200)
    cv.imshow(img, canny)
    cv.waitKey(0)
    cv.destroyWindow(img)